In [ ]:
import requests

from sandag_rsm.data_load.zones import load_mgra_data
from sandag_rsm.data_load.triplist import load_trip_list, trip_mode_shares_by_mgra, trip_mode_shares_by_taz
from sandag_rsm.poi import poi_taz_mgra, attach_poi_taz_skims
from sandag_rsm.zone_agg import aggregate_zones, viewer, viewer2, aggregate_zones_within_districts, merge_zone_data

from sandag_rsm.translate import translate_demand
import os
import numpy as np
import pandas as pd
import geopandas as gpd

## Remote I/O

In [ ]:
data_dir = "../test/data/"
export_dir = "../test/export/"
resource_url = "https://media.githubusercontent.com/media/wsp-sag/client_sandag_rsm_resources/main/"

In [ ]:
skim_filename = "traffic_skims_AM_mini.omx"
matrix_name = 'AM_SOV_TR_M_TIME'

In [ ]:
## Create Dummy 'zone_to_cluster' Dictionary
zone_to_cluster = {i:1+i%100 for i in range(4996)}

## Demo

In [ ]:
matrix_load, matrix = translate_demand(skim_filename, matrix_name, zone_to_cluster, data_dir, export_dir)

In [ ]:
print('Loaded Matrix')
print(f'total sum: {matrix_load.sum().sum():,.0f}\n')
print(matrix_load.sum().describe().apply("{0:,.2f}".format))

In [ ]:
print('Aggregated Matrix')
print(f'total sum: {matrix.sum().sum():,.0f}\n')
print(matrix.sum().describe().apply("{0:,.2f}".format))

In [ ]:
matrix_load

In [ ]:
matrix

## Visualize

In [ ]:
use_cluster = 731
use_col = f'{matrix_name}_{use_cluster}'

plot_col = matrix[use_cluster].reset_index()
plot_col.columns = ['cluster_id', use_col]

In [ ]:
mgra = gpd.read_file(os.path.join(data_dir, 'MGRASHAPE.zip'))
mgra_xref = pd.read_csv(os.path.join(data_dir, 'mgra_crosswalk.csv'))

clusters = pd.merge(mgra, mgra_xref, on='MGRA', how='left')
clusters = clusters[['cluster_id', 'geometry']].dissolve(by='cluster_id').reset_index()

clusters = pd.merge(clusters, plot_col, on='cluster_id', how='left')

In [ ]:
viewer(clusters, color=use_col, marker_line_width=0)